In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import pickle
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, f1_score
from torch.utils.data import DataLoader, TensorDataset

# 1. TabTransformer 구조 (동일)
class TabTransformer(nn.Module):
    def __init__(self, input_dim, embed_dim=32, num_heads=8, num_layers=4):
        super(TabTransformer, self).__init__()
        self.numerical_projection = nn.Linear(input_dim, input_dim * embed_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.mlp = nn.Sequential(
            nn.Linear(input_dim * embed_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.numerical_projection(x).view(x.size(0), -1, 32)
        x = self.transformer(x)
        x = x.flatten(1)
        return self.mlp(x)

# 2. 메인 실행 함수
def run_tabtransformer_with_checkpoints():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"✅ Using device: {device}")
    
    test_df = pd.read_csv("test_smote.csv")
    X_test_all = torch.FloatTensor(test_df.drop('Class', axis=1).values).to(device)
    y_test_all = test_df['Class'].values

    dataset_configs = {
        "Original": "base_preprocessed.csv",
        "SMOTE": "train_smote.csv",
        "cGAN": "train_cgan.csv"
    }

    for name, filename in dataset_configs.items():
        if not os.path.exists(filename):
            continue
            
        print(f"\n🚀 [{name}] 학습 시작 (Total 100 Epochs)")
        df = pd.read_csv(filename)
        
        if name == "Original":
            train_df, _ = train_test_split(df, test_size=0.2, random_state=42, stratify=df['Class'])
        else:
            train_df = df

        X_tr = torch.FloatTensor(train_df.drop('Class', axis=1).values).to(device)
        y_tr = torch.FloatTensor(train_df['Class'].values).view(-1, 1).to(device)
        train_loader = DataLoader(TensorDataset(X_tr, y_tr), batch_size=1024, shuffle=True)

        model = TabTransformer(input_dim=X_tr.shape[1]).to(device)
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.BCELoss()

        # --- 최적 모델 추적용 변수 ---
        best_f1 = 0.0
        best_state_dict = None
        best_threshold = 0.5

        model.train()
        for epoch in range(1, 101):
            epoch_loss = 0
            for batch_x, batch_y in train_loader:
                optimizer.zero_grad()
                output = model(batch_x)
                loss = criterion(output, batch_y)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
            
            # 10에포크마다 체크포인트 및 성능 검증
            if epoch % 10 == 0:
                model.eval()
                with torch.no_grad():
                    y_probs = model(X_test_all).cpu().numpy().flatten()
                
                # 임계값 및 F1 계산
                precisions, recalls, thresholds = precision_recall_curve(y_test_all, y_probs)
                f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-10)
                current_best_idx = np.argmax(f1_scores)
                current_f1 = f1_scores[current_best_idx]
                current_th = thresholds[current_best_idx] if current_best_idx < len(thresholds) else 0.5

                print(f"📊 Epoch {epoch:3d} | Loss: {epoch_loss/len(train_loader):.4f} | Test F1: {current_f1:.4f}")

                # 최적 모델 갱신 (가장 높은 F1 기준)
                if current_f1 > best_f1:
                    best_f1 = current_f1
                    best_state_dict = model.state_dict().copy() # 현재 가중치 복사
                    best_threshold = current_th
                    print(f"    🌟 New Best Model Found!")

                # 체크포인트 파일 저장 (물리적 백업)
                cp_filename = f"checkpoint_{name.lower()}_epoch_{epoch}.pt"
                torch.save(model.state_dict(), cp_filename)
                
                model.train() # 다시 학습 모드로

        # 4. 최종 Best 모델을 PKL로 저장 (Airflow 배포용)
        save_filename = f"tabtransformer_model_{name.lower()}.pkl"
        model_metadata = {
            "model_type": "TabTransformer",
            "augmentation": name,
            "input_dim": X_tr.shape[1],
            "model_state_dict": best_state_dict, # 100번 에포크 중 최고 성능 가중치
            "threshold": best_threshold,
            "feature_names": list(train_df.drop('Class', axis=1).columns),
            "best_f1": best_f1
        }

        with open(save_filename, "wb") as f:
            pickle.dump(model_metadata, f)
            
        print(f"✅ [{name}] 저장 완료! (Best F1: {best_f1:.4f} at Threshold: {best_threshold:.4f})")

if __name__ == "__main__":
    run_tabtransformer_with_checkpoints()

✅ Using device: cpu


KeyboardInterrupt: 